#**Programa para asignar curules de representación proporcional (RP) Veracruz utilizando como base la elección de 2021**


###**1.- Importamos las librerias que se van a utilizar**

In [64]:
# Importamos los paquetes a utilizar
import pandas as pd
import os
import numpy as np
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [65]:
# Cargamos a las bases a utilizar
root_path = "drive/MyDrive/RP_Veracruz_2021"
orig = root_path + "/original_db"
os.listdir(orig)

['RP_veracruz_2021.csv']

###**2.- Generamos un dataframe con los datos de las votaciones 2021**

In [66]:
#Creamos un dataframe con los datos de las votaciones de 2021
df_rp = pd.read_csv(orig + "/RP_veracruz_2021.csv")
#Calculamos la votación total
total_votos = df_rp['Votación'].sum()
#Agregamos las filas con la suma de la votación total al dataframe
df_rp.loc[len(df_rp.index)] = ['Votación total', total_votos]
df_rp

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
7,TxV,63903
8,PODEMOS,66414
9,P. CARDENISTA,27509


###**3.- Creamos un filtro para con la votación válida emitida y calculamos el umbral de 3% de acuerdo con la Ley Electoral de Veracruz**

In [67]:
#Calculamos la votación válida emitida restando votos nulos (VN) y candidatos no registrados (CNR)
votos_nulos = df_rp.loc[df_rp['Partido'] == 'VN', 'Votación'].values[0]
candidatos_no_registrados = df_rp.loc[df_rp['Partido'] == 'CNR', 'Votación'].values[0]
votacion_valida_emitida = total_votos - votos_nulos - candidatos_no_registrados
#Agregamos la fila con la votación válida emitida al dataframe
df_rp.loc[len(df_rp.index)] = ['Votación válida emitida', votacion_valida_emitida]
#Creamos un filtro del dataframe para excluir CNR, VN y Votación total
part_vve = df_rp[~df_rp['Partido'].isin(['CNR', 'VN', 'Votación total'])]
#Dataframe filtrado
part_vve

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
7,TxV,63903
8,PODEMOS,66414
9,P. CARDENISTA,27509


In [68]:
#Calculamos el umbral del 3% para determinar aquellos partidos que cumplen dicho criterio
umbral_3 = votacion_valida_emitida * 0.03

###**4.- Calculamos la votación estatal válida emitida para aquellos partidos que cumplen con el 3%**

In [69]:
#Filtramos partidos que cumplen con el 3% de votación válida emitida
part_veve = df_rp[(df_rp['Votación'] >= umbral_3) & ~df_rp['Partido'].isin(['CNR', 'VN', 'Votación total', 'Votación válida emitida'])]
#Calculamos la suma de la votación de los partidos que cumplen con el umbral del 3%
votacion_estatal_valida_emitida = part_veve['Votación'].sum()
#Agregamos la fila con la votación estatal válida emitida al dataframe filtrado
part_veve.loc[len(part_veve.index)] = ['Votación estatal válida emitida', votacion_estatal_valida_emitida]
part_veve

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
13,FxM,110605
8,Votación estatal válida emitida,3134284


###**5.- Calculamos el cociente natural de acuerdo con la ley de Veracruz dividiendo la votación estatal válida emitida entre el total de escaños de RP a repartir**

In [70]:
#Número total de escaños a repartir de acuerdo con la ley electoral de Veracruz
total_escanos = 20
#Calculamos el cociente natural
cociente_natural = votacion_estatal_valida_emitida / total_escanos
cociente_natural_rounded = round(cociente_natural, 1)
print(f'Cociente natural: {cociente_natural_rounded}')

Cociente natural: 156714.2


In [71]:
#Eliminamos la fila de "Votación estatal válida emitida"
part_veve = part_veve[part_veve['Partido'] != 'Votación estatal válida emitida']

###**6.- Paso 1: Asignación por cociente natural**

In [72]:
#Paso 1: Asignamos por Cociente Natural (CN)
part_veve['Escanos_CN'] = (part_veve['Votación'] // cociente_natural).astype(int)
#Añadimos la columna del cociente natural para impresión
part_veve['Cociente_Natural'] = cociente_natural_rounded
# Imprimimos resultados por cociente natural excluyendo "Votación estatal válida emitida"
part_veve[['Partido', 'Votación', 'Cociente_Natural', 'Escanos_CN']]
# Podemos observar que se asignan 16 curules dividiendo la votación estatal emitida y el cociente natural

,Partido,Votación,Cociente_Natural,Escanos_CN
0,PAN,542116,156714.2,3
1,PRI,367436,156714.2,2
2,PRD,204365,156714.2,1
3,VERDE,213508,156714.2,1
4,PT,129147,156714.2,0
5,MC,255904,156714.2,1
6,MORENA,1311203,156714.2,8
13,FxM,110605,156714.2,0


###**7.- Paso 2: Asignación por resto mayor**

In [73]:
# Paso 2: Asignamos por Resto Mayor (RM)
# Número de escaños restantes
Escanos_RM = total_escanos - part_veve['Escanos_CN'].sum()
print(f'Escanos RM: {Escanos_RM}')

Escanos RM: 4


In [74]:
#Calculamos los votos utilizados y el resto mayor
part_veve['Votos_utilizados'] = part_veve['Escanos_CN'] * cociente_natural
part_veve['Resto_mayor'] = part_veve['Votación'] - part_veve['Votos_utilizados']
#Ordenamos los partidos por resto mayor en orden descendente
part_veve = part_veve.sort_values(by='Resto_mayor', ascending=False)
#Asignamos escaños restantes a los partidos con el mayor resto mayor
part_veve['Escanos_RM'] = 0
part_veve.iloc[:Escanos_RM, part_veve.columns.get_loc('Escanos_RM')] = 1
#Sumamos los escaños CN y los escaños RM para obtener la asignación final de los 20 escaños
part_veve['Escanos_totales'] = part_veve['Escanos_CN'] + part_veve['Escanos_RM']
part_veve[['Partido', 'Votos_utilizados', 'Resto_mayor', 'Escanos_CN', 'Escanos_RM', 'Escanos_totales']]

,Partido,Votos_utilizados,Resto_mayor,Escanos_CN,Escanos_RM,Escanos_totales
4,PT,0.0,129147.0,0,1,1
13,FxM,0.0,110605.0,0,1,1
5,MC,156714.2,99189.8,1,1,2
0,PAN,470142.6,71973.4,3,1,4
6,MORENA,1253713.6,57489.4,8,0,8
3,VERDE,156714.2,56793.8,1,0,1
1,PRI,313428.4,54007.6,2,0,2
2,PRD,156714.2,47650.8,1,0,1
